# Simple Web Proxy

In [2]:
import os
import re
import sys
import base64
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html

In [ ]:
sudoPassword = input("Password: ")

In [ ]:
def render(source_url):
    
    from PyQt5.QtWidgets import QApplication
    from PyQt5.QtCore import QUrl
    from PyQt5.QtWebEngineWidgets import QWebEngineView

    class Render(QWebEngineView):
        def __init__(self, url):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            #self.setHtml(html)
            self.load(QUrl(url))
            self.app.exec_()

        def _loadFinished(self, result):
        # This is an async call, you need to wait for this
        # to be called before closing the app
            self.page().toHtml(self._callable)

        def _callable(self, data):
            self.html = data
            # Data has been stored, it's safe to quit the app
            self.app.quit()

    return Render(source_url).html

In [5]:
print("Current Proxy Settings:\n")
os.system("networksetup -getsecurewebproxy Wi-Fi")
print("\n\nChoose option:")
var = int(input("\n\t(1) GB\n\t(2) DE\n\t(3) AT\n\t(4) Other\n\n\t(9) SWITCH OFF\n\t(0) exit\n"))

Current Proxy Settings:



Choose option:

	(1) GB
	(2) DE
	(3) AT
	(4) Other

	(9) SWITCH OFF
	(0) exit
3


In [6]:
if var > 0 and var < 4:
	if var == 1:
		url = "http://spys.one/free-proxy-list/GB/"
	if var == 2:
		url = "http://spys.one/free-proxy-list/DE/"
	if var == 3:
		url = "http://spys.one/free-proxy-list/AT/"
	if var == 4:
		url = "http://spys.one/en/https-ssl-proxy/"

	r = render(url)

In [7]:
soup = BeautifulSoup(r, "lxml")

In [8]:
table = soup.find_all('table')[1].find('table')
rows = table.find_all('tr', {'class': re.compile(r'^spy')})[3:]  #rows = table.find_all('tr')[3:]
rows = rows[:-1] # remove last row

In [9]:
# headers of dataframe
data = {
	'Address' : [],
	'Type' : [],
	'Anonymity' : [],
	'Country' : [],
	'Name' : [],
	'Latency' : [],
	'Speed' : [],
	'Uptime' : [],
	'Checkdate' : []
	}
print(data)

{'Address': [], 'Type': [], 'Anonymity': [], 'Country': [], 'Name': [], 'Latency': [], 'Speed': [], 'Uptime': [], 'Checkdate': []}


In [10]:
# filling columns
for row in rows:
    cols = row.find_all('td')
    data['Address'].append(cols[0].get_text())
    data['Type'].append(cols[1].get_text())
    data['Anonymity'].append(cols[2].get_text())
    data['Country'].append(cols[3].get_text())
    data['Name'].append(cols[4].get_text())
    data['Latency'].append(cols[5].get_text())
    data['Speed'].append(cols[6].get_text())
    data['Uptime'].append(cols[7].get_text())
    data['Checkdate'].append(cols[8].get_text())

In [11]:
proxies2 = pd.DataFrame(data)
proxies2 = proxies2[proxies2.Type.str.contains("HTTPS") == True]
proxies = proxies2.sort_values(by='Latency')
proxies = proxies.reset_index()
proxies.to_csv("proxylist.csv")

In [12]:
# proxy address
one = proxies.at[0, 'Address']
country = proxies.at[0, 'Country']
proxy = re.findall( r'[0-9]+(?:\.[0-9]+){3}', one)
proxy = str(proxy[0])
print(proxy)

185.7.19.173


In [13]:
# port number
port = one.split(':')[-1]
print(port)

8080


In [14]:
print("\n\tAddress: " + proxy + " \t Port: " + port + " \t Country: " + country)


	Address: 185.7.19.173 	 Port: 8080 	 Country: Austria (Imst)


In [15]:
command = "networksetup -setsecurewebproxy Wi-Fi " + proxy + " " + port
os.system('echo %s|sudo -S %s' % (sudoPassword, command))
command = "networksetup -setwebproxy Wi-Fi " + proxy + " " + port
os.system('echo %s|sudo -S %s' % (sudoPassword, command))

0

In [16]:
os.system("networksetup -getsecurewebproxy Wi-Fi")

0